# Create database

In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Connect to an existing database
conn = psycopg2.connect("host=postgres dbname=postgres port=5432 user=postgres password=changeme")

# Open a cursor to perform database operations
cur = conn.cursor()
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

cur.execute("CREATE DATABASE movies")
conn.commit()
    
# Close communication with the database
cur.close()
conn.close()

# Reconnect, this time to movies database

In [2]:
import psycopg2

# Connect to an existing database
conn = psycopg2.connect("host=postgres dbname=movies port=5432 user=postgres password=changeme")

# Open a cursor to perform database operations
cur = conn.cursor()

# Create Tables with relations

In [3]:
cur.execute("""
        -- Table: public.movies

        -- DROP TABLE IF EXISTS public.movies;

        CREATE TABLE IF NOT EXISTS public.movies
        (
            "movieId" integer NOT NULL,
            title character varying COLLATE pg_catalog."default",
            genres character varying COLLATE pg_catalog."default",
            CONSTRAINT movies_pkey PRIMARY KEY ("movieId")
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.movies
            OWNER to postgres;
            """)
conn.commit()

In [4]:
cur.execute("""
        -- Table: public.links

        -- DROP TABLE IF EXISTS public.links;

        CREATE TABLE IF NOT EXISTS public.links
        (
            "movieId" integer NOT NULL,
            "imdbId" integer NOT NULL,
            "tmdbId" integer,
            CONSTRAINT links_pkey PRIMARY KEY ("movieId"),
            CONSTRAINT "movieId_movies" FOREIGN KEY ("movieId")
                REFERENCES public.movies ("movieId") MATCH SIMPLE
                ON UPDATE NO ACTION
                ON DELETE NO ACTION
                NOT VALID
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.links
            OWNER to postgres;
            """)
conn.commit()

In [5]:
cur.execute("""
        -- Table: public.ratings

        -- DROP TABLE IF EXISTS public.ratings;

        CREATE TABLE IF NOT EXISTS public.ratings
        (
            "userId" integer NOT NULL,
            "movieId" integer NOT NULL,
            rating numeric NOT NULL,
            "timestamp" integer NOT NULL,
            CONSTRAINT ratings_pkey PRIMARY KEY ("userId", "movieId"),
            CONSTRAINT "movieId_ratings" FOREIGN KEY ("movieId")
                REFERENCES public.links ("movieId") MATCH SIMPLE
                ON UPDATE NO ACTION
                ON DELETE NO ACTION
                NOT VALID
        )

        TABLESPACE pg_default;

        ALTER TABLE IF EXISTS public.ratings
            OWNER to postgres;
            """)
conn.commit()

In [6]:
cur.execute("""
    -- Table: public.tags

    -- DROP TABLE IF EXISTS public.tags;

    CREATE TABLE IF NOT EXISTS public.tags
    (
        "userId" integer,
        "movieId" integer NOT NULL,
        tag character varying COLLATE pg_catalog."default",
        "timestamp" integer,
        CONSTRAINT "movieId_tags" FOREIGN KEY ("movieId")
            REFERENCES public.links ("movieId") MATCH SIMPLE
            ON UPDATE NO ACTION
            ON DELETE NO ACTION
            NOT VALID
    )

    TABLESPACE pg_default;

    ALTER TABLE IF EXISTS public.tags
        OWNER to postgres;
            """)
conn.commit()

# Add Data to tables

In [7]:
cur.execute("""
    COPY movies FROM '/db/ml-latest-small/movies.csv' DELIMITER ',' CSV HEADER;
""")

conn.commit()

In [8]:
cur.execute("""
    COPY links FROM '/db/ml-latest-small/links.csv' DELIMITER ',' CSV HEADER;
""")

conn.commit()

In [9]:
cur.execute("""
    COPY ratings FROM '/db/ml-latest-small/ratings.csv' DELIMITER ',' CSV HEADER;
""")

conn.commit()

In [10]:
cur.execute("""
    COPY tags FROM '/db/ml-latest-small/tags.csv' DELIMITER ',' CSV HEADER;
""")

conn.commit()

In [11]:
# Close communication with the database
cur.close()
conn.close()

In [1]:
print("Database Initiation done")

Done


In [ ]:
import psycopg2

# Connect to an existing database
conn = psycopg2.connect("host=postgres port=5432 dbname=movies user=postgres password=changeme")

# Open a cursor to perform database operations
cur = conn.cursor()

# TASK 1
How many movies are in data set ?

In [ ]:
cur.execute("SELECT COUNT(*) FROM movies;")
cur.fetchone()

Answer: (number of movies in dataset)

# TASK 2 - NOT DONE
What is the most common genre of movie?
NOT DONE

# TASK 3
What are top 10 movies with highest rate ?

In [ ]:
cur.execute(
    """SELECT "title", ROUND(AVG("rating"), 2)
    FROM movies
        JOIN ratings USING ("movieId")
    GROUP BY "movieId"
    ORDER BY AVG("rating") DESC
    LIMIT 10;"""
)
cur.fetchall()

Answer: (title, average rating)

# TASK 4 
What are 5 most often rating users ?

In [ ]:
cur.execute(
    """SELECT "userId",
     COUNT("userId")
    FROM ratings
    GROUP BY  "userId"
    ORDER BY COUNT("userId") DESC
    LIMIT 10;"""
)
cur.fetchall()

Answer(userID, number of ratings)

# TASK 5
When was done first and last rate included in data set and what was the rated movie tittle?

In [ ]:
from datetime import datetime

cur.execute(
    """SELECT MIN("timestamp"), MAX("timestamp")
    FROM ratings"""
)
result = cur.fetchall()

oldest_rating = datetime.utcfromtimestamp(result[0][0]).strftime('%Y-%m-%d %H:%M:%S')
newest_rating = datetime.utcfromtimestamp(result[0][1]).strftime('%Y-%m-%d %H:%M:%S')

print(f"Oldest rating was created on {oldest_rating}")
print(f"Newest rating was created on {newest_rating}")


# TASK 6
Find all movies released in 1990

In [ ]:
cur.execute(
    "SELECT title FROM movies WHERE title LIKE '%(1990)%'"
)
result = cur.fetchall()
# print(result)
for i in result:
    print(i[0].removesuffix(' (1990)'))

In [ ]:
# Close communication with the database
cur.close()
conn.close()